In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import shutil
import time

# Step 1 - Scraping

## NASA Mars News

In [2]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'



In [3]:
response = requests.get(url)

In [4]:
soup = bs(response.text, 'html.parser')

In [5]:
results = soup.find('div', class_ = 'slide')
print(results.prettify())

<div class="slide">
 <div class="image_and_description_container">
  <a href="/news/8436/why-this-martian-full-moon-looks-like-candy/">
   <div class="rollover_description">
    <div class="rollover_description_inner">
     For the first time, NASA's Mars Odyssey orbiter has caught the Martian moon Phobos during a full moon phase. Each color in this new image represents a temperature range detected by Odyssey's infrared camera.
    </div>
    <div class="overlay_arrow">
     <img alt="More" src="/assets/overlay-arrow.png"/>
    </div>
   </div>
   <img alt="Why This Martian Full Moon Looks Like Candy" class="img-lazy" data-lazy="/system/news_items/list_view_images/8436_PIA23205_annotated-32x24.jpg" src="/assets/loading_320x240.png"/>
  </a>
 </div>
 <div class="content_title">
  <a href="/news/8436/why-this-martian-full-moon-looks-like-candy/">
   Why This Martian Full Moon Looks Like Candy
  </a>
 </div>
</div>



In [6]:
content_title = soup.find_all('div', class_ = "content_title")
title = []

for ct in content_title:
    if (ct.a):
         if (ct.a.text):
                title.append(ct.text)


    title_text = title[0]
    title_text = title_text.replace("\n\n", "")
    
print(title_text)

Why This Martian Full Moon Looks Like Candy


In [7]:
description = soup.find('div', class_ = "rollover_description_inner")
print(description.prettify())

<div class="rollover_description_inner">
 For the first time, NASA's Mars Odyssey orbiter has caught the Martian moon Phobos during a full moon phase. Each color in this new image represents a temperature range detected by Odyssey's infrared camera.
</div>



In [8]:
desc = []
for dc in description:
    if (dc):
        desc.append(dc)
        
    description_ = dc.replace("\n","")
print(description_)
        


For the first time, NASA's Mars Odyssey orbiter has caught the Martian moon Phobos during a full moon phase. Each color in this new image represents a temperature range detected by Odyssey's infrared camera.


## JPL Mars Space Images - Featured Image

In [6]:
# Use splinter to navigate the site and find the image url 
# for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
# Make sure to find the image url to the full size .jpg image.
# Make sure to save a complete url string for this image.
# https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars

from splinter import Browser
!which chromedriver
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)
time.sleep(1)

/usr/local/bin/chromedriver


In [11]:
# Visit Mars Space Images through splinter module
url_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_image)
time.sleep(1)

In [ ]:
xpath = "//*[@id=\"page\"]/section[3]/div/ul/li[3]/a/div/div[2]/img"

In [12]:
results = browser.find_by_xpath(xpath)
img = results[0]
img.click()
time.sleep(2)

In [13]:
html_img =browser.html
soup = bs(html_img, "html.parser")
img_url = soup.find('img', class_='fancybox-image')["src"]

In [14]:
from urllib.parse import urlsplit
base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url_image))
print(base_url)

https://www.jpl.nasa.gov/


In [15]:
featured_image_url = base_url + img_url
print(featured_image_url)

https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA23140_hires.jpg


## Mars Weather

In [16]:
# Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. 
# Save the tweet text for the weather report as a variable called mars_weather.
from splinter import Browser
!which chromedriver
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)
time.sleep(1)

url_weather = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_weather)
time.sleep(1)

/usr/local/bin/chromedriver


In [17]:
html_weather = browser.html
soup = bs(html_weather, "html.parser")
mars_weather = soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
time.sleep(2)

In [18]:
print(mars_weather)

InSight sol 138 (2019-04-17) low -97.7ºC (-143.9ºF) high -17.3ºC (0.9ºF)
winds from the W at 4.3 m/s (9.5 mph) gusting to 12.6 m/s (28.1 mph)
pressure at 7.30 hPapic.twitter.com/ofMyPZM2vS


In [19]:
mars_weather = mars_weather.split('hPa')[0]

In [20]:
mars_weather = mars_weather.replace("\n", " ")

In [21]:
import re
patterns = ['r"http\S+"', 'r"\xa0pic\S+"', 'r"pic\S+"', 'r"pbs\S+"']
for pattern in patterns:
    mars_weather = re.sub(pattern, "", mars_weather).strip()
print(mars_weather)

InSight sol 138 (2019-04-17) low -97.7ºC (-143.9ºF) high -17.3ºC (0.9ºF) winds from the W at 4.3 m/s (9.5 mph) gusting to 12.6 m/s (28.1 mph) pressure at 7.30


## Mars Facts

In [22]:
url_facts = "https://space-facts.com/mars/"

In [23]:
table = pd.read_html(url_facts)

In [24]:
table_df = pd.DataFrame(table[0])
table_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [25]:
table_df.columns=['Index', 'Values']

In [26]:
table_df = table_df.set_index('Index')

In [27]:
table_df

,Values
Index,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [28]:
table_df = table_df.to_html()
table_df = table_df.replace("\n", " ")

In [29]:
table_df

'<table border="1" class="dataframe">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Values</th>     </tr>     <tr>       <th>Index</th>       <th></th>     </tr>   </thead>   <tbody>     <tr>       <th>Equatorial Diameter:</th>       <td>6,792 km</td>     </tr>     <tr>       <th>Polar Diameter:</th>       <td>6,752 km</td>     </tr>     <tr>       <th>Mass:</th>       <td>6.42 x 10^23 kg (10.7% Earth)</td>     </tr>     <tr>       <th>Moons:</th>       <td>2 (Phobos &amp; Deimos)</td>     </tr>     <tr>       <th>Orbit Distance:</th>       <td>227,943,824 km (1.52 AU)</td>     </tr>     <tr>       <th>Orbit Period:</th>       <td>687 days (1.9 years)</td>     </tr>     <tr>       <th>Surface Temperature:</th>       <td>-153 to 20 °C</td>     </tr>     <tr>       <th>First Record:</th>       <td>2nd millennium BC</td>     </tr>     <tr>       <th>Recorded By:</th>       <td>Egyptian astronomers</td>     </tr>   </tbody> </table>'

## Mars Hemispheres

1. Visit the USGS Astrogeology site here to obtain high resolution 
   images for each of Mar's hemispheres.
    
2. You will need to click each of the links to the hemispheres
   in order to find the image url to the full resolution image.
    
3. Save both the image url string for the full resolution 
   hemisphere image, and the Hemisphere title containing 
   the hemisphere name. Use a Python dictionary to store 
   the data using the keys img_url and title.
    
4. Append the dictionary with the image url string and the 
   hemisphere title to a list. This list will contain 
   one dictionary for each hemisphere.

In [30]:
from splinter import Browser
!which chromedriver
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

/usr/local/bin/chromedriver


In [32]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
base_url = 'https://astrogeology.usgs.gov'
# get page with request module
response = requests.get(url)
soup = bs(response.text, 'html.parser')
# print(soup.prettify())

#scrape one link on the home page
#links = soup.find(class_="item",).find('a')['href']
links = soup.find_all(class_="item")
urls = []
for link in links:
    urls.append(f'{base_url}'+link.a['href'])
    
hemisphere_image_urls = []
for url in urls:
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    time.sleep(2)
    title = soup.find(class_="title").text
    image = soup.find(class_="downloads").a['href']
    hem_dict = {'title': title, 'img_url': image}
    hemisphere_image_urls.append(hem_dict)
    
browser.quit()
hemisphere_image_urls


[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [33]:
print(description_)
print("-"*50)
print(featured_image_url)
print("-"*50)
print(mars_weather)
print("-"*50)
print(table_df)

Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.
--------------------------------------------------
https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA23140_hires.jpg
--------------------------------------------------
InSight sol 138 (2019-04-17) low -97.7ºC (-143.9ºF) high -17.3ºC (0.9ºF) winds from the W at 4.3 m/s (9.5 mph) gusting to 12.6 m/s (28.1 mph) pressure at 7.30
--------------------------------------------------
<table border="1" class="dataframe">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Values</th>     </tr>     <tr>       <th>Index</th>       <th></th>     </tr>   </thead>   <tbody>     <tr>       <th>Equatorial Diameter:</th>       <td>6,792 km</td>     </tr>     <tr>       <th>Polar Diameter:</th>       <td>6,752 km</td>     </tr>     <tr>       <th>Mass:</th>       <td>6.42 x 10^23 kg (10.7% Earth)</td>     </tr>     

In [8]:
# for loop to get path for each imgs
# xpath1 = "//*[@id='product-section']/div[2]/div[i]/a/img"
mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(mars_hemisphere_url)
time.sleep(1)
 

xpath1 = []
hemi_url = []

# Get path for each hemisphere website
for i in range(1,5):
    x = f"//*[@id='product-section']/div[2]/div[{i}]/a/img"

    
    
    results = browser.find_by_xpath(x)
    img = results[0]
    img.click()
    time.sleep(1)

    xpath3 = "//*[@id='wide-image']/div/ul/li/a"
    results = browser.find_by_xpath(xpath3)
    img = results[0]
    img.click()
    time.sleep(1)
    
    html = browser.html
    soup = bs(html, 'html.parser')
    url = soup.find("img")["src"]
    hemi_url.append(url)
    
    
#     browser.windows.current = browser.windows[-1]
#     hemi_img_html = browser.html
#     browser.windows.current = browser.windows[0]
#     browser.windows[-1].close()

    
    
    url_hemisphere = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    browser.visit(url_hemisphere)